In [2]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


In [3]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git

  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-1424nauz
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-1424nauz
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit 5741c522547756ac4bb7a16df32106a15efb8a57
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for nvcc4jupyter: filename=nvcc4jupyter-1.2.1-py3-none-any.whl size=10741 sha256=1c43b610d84440f376c57bb0b3d20f87e2433fdb6f06eb98d5fd7c81c461a971
  Stored in directory: /tmp/pip-ephem-wheel-cache-nhehoqnu/wheels/a8/b9/18/23f8ef71ceb0f63297dd1903aedd067e6243a68ea756d6feea
Successfully built nvcc4jupyter


In [ ]:
#Vector Addition:

%%writefile vector_Addition_2.cu
#include<iostream>
#include<cuda.h>
#include<cuda_runtime.h>
#define BLOCK_SIZE 256  // Optimal for GPU usage

using namespace std;

void fill_array(int *arr, int size) {
    for (int i = 0; i < size; i++) {
        arr[i] = rand() % 100;
    }
}

void add_cpu(int *arr1, int *arr2, int *result, int size) {
    for (int i = 0; i < size; i++) {
        result[i] = arr1[i] + arr2[i];
    }
}

void print_matrix(int *arr, int size) {
    for (int i = 0; i < size; i++) {
        cout << arr[i] << " ";
    }
    cout << endl;
}

__global__ void add(int *arr1, int *arr2, int *arr3, int size) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < size) {
        arr3[idx] = arr1[idx] + arr2[idx];
    }
}

int main() {
    int size;
    cout << "Enter size of vector: ";
    cin >> size;
    int *arr1_cpu = new int[size], *arr2_cpu = new int[size], *result_cpu = new int[size];

    fill_array(arr1_cpu, size);
    cout << "Array 1: ";
    print_matrix(arr1_cpu, size);
    fill_array(arr2_cpu, size);
    cout << "Array 2: ";
    print_matrix(arr2_cpu, size);

    int *arr1_gpu, *arr2_gpu, *result_gpu;
    cudaMalloc(&arr1_gpu, size * sizeof(int));
    cudaMalloc(&arr2_gpu, size * sizeof(int));
    cudaMalloc(&result_gpu, size * sizeof(int));

    cudaMemcpy(arr1_gpu, arr1_cpu, size * sizeof(int), cudaMemcpyHostToDevice);
    cudaMemcpy(arr2_gpu, arr2_cpu, size * sizeof(int), cudaMemcpyHostToDevice);

    // GPU timing
    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);
    cudaEventRecord(start);

    dim3 dimGrid((size + BLOCK_SIZE - 1) / BLOCK_SIZE), dimBlock(BLOCK_SIZE);
    add<<<dimGrid, dimBlock>>>(arr1_gpu, arr2_gpu, result_gpu, size);

    cudaEventRecord(stop);
    cudaEventSynchronize(stop);
    float gpuTime = 0;
    cudaEventElapsedTime(&gpuTime, start, stop);

    cudaMemcpy(result_cpu, result_gpu, size * sizeof(int), cudaMemcpyDeviceToHost);

    cout << "GPU result:\n";
    print_matrix(result_cpu, size);
    cout << "GPU Elapsed Time = " << gpuTime << " milliseconds" << endl;

    // CPU timing
    cudaEventRecord(start);
    add_cpu(arr1_cpu, arr2_cpu, result_cpu, size);
    cudaEventRecord(stop);
    cudaEventSynchronize(stop);
    float cpuTime = 0;
    cudaEventElapsedTime(&cpuTime, start, stop);

    cout << "CPU result:\n";
    print_matrix(result_cpu, size);
    cout << "CPU Elapsed Time = " << cpuTime << " milliseconds" << endl;

    cudaEventDestroy(start);
    cudaEventDestroy(stop);
    cudaFree(arr1_gpu);
    cudaFree(arr2_gpu);
    cudaFree(result_gpu);
    delete[] arr1_cpu;
    delete[] arr2_cpu;
    delete[] result_cpu;

    return 0;
}


In [ ]:
!nvcc -o vector_Addition_2 vector_Addition_2.cu
!./vector_Addition_2

In [11]:
# Matrix Mul

%%writefile matmul.cu
#include <iostream>
#include <cuda.h>
#include <cuda_runtime.h>
#define BLOCK_SIZE 256  // Suitable for many devices; consider your GPU's specs for optimization

using namespace std;

// Function to initialize matrix with random values
void initialize_matrix(int *array, int rows, int cols) {
    for (int i = 0; i < rows; i++) {
        for (int j = 0; j < cols; j++) {
            array[i * cols + j] = rand() % 10;  // Keeping numbers small for easy visualization
        }
    }
}

// Function to print matrix
/*void print_matrix(int *array, int rows, int cols) {
    for (int i = 0; i < rows; i++) {
        for (int j = 0; j < cols; j++) {
            cout << array[i * cols + j] << " ";
        }
        cout << endl;
    }
}*/

// CPU matrix multiplication
void matrix_multiplication_cpu(int *a, int *b, int *c, int common, int c_rows, int c_cols) {
    for (int i = 0; i < c_rows; i++) {
        for (int j = 0; j < c_cols; j++) {
            int sum = 0;
            for (int k = 0; k < common; k++) {
                sum += a[i * common + k] * b[k * c_cols + j];
            }
            c[i * c_cols + j] = sum;
        }
    }
}

// GPU kernel for matrix multiplication
__global__ void matrix_multiply(int *a, int *b, int *c, int common, int c_rows, int c_cols) {
    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int col = blockIdx.x * blockDim.x + threadIdx.x;
    if (row < c_rows && col < c_cols) {
        int sum = 0;
        for (int k = 0; k < common; k++) {
            sum += a[row * common + k] * b[k * c_cols + col];
        }
        c[row * c_cols + col] = sum;
    }
}

int main() {
    int A_rows, A_cols, B_cols;
    cout << "Dimensions of matrix 1:\nRows: ";
    cin >> A_rows;
    cout << "Columns: ";
    cin >> A_cols;
    cout << "Dimensions of matrix 2:\nRows: " << A_cols << "\nColumns: ";
    cin >> B_cols;

    int B_rows = A_cols, C_rows = A_rows, C_cols = B_cols;
    int *A = new int[A_rows * A_cols];
    int *B = new int[B_rows * B_cols];
    int *C = new int[C_rows * C_cols];

    initialize_matrix(A, A_rows, A_cols);
    /*cout << "Matrix 1\n";
    print_matrix(A, A_rows, A_cols);*/

    initialize_matrix(B, B_rows, B_cols);
    /* cout << "Matrix 2\n";
    print_matrix(B, B_rows, B_cols);*/

    int *d_A, *d_B, *d_C;
    cudaMalloc(&d_A, A_rows * A_cols * sizeof(int));
    cudaMalloc(&d_B, B_rows * B_cols * sizeof(int));
    cudaMalloc(&d_C, C_rows * C_cols * sizeof(int));

    cudaMemcpy(d_A, A, A_rows * A_cols * sizeof(int), cudaMemcpyHostToDevice);
    cudaMemcpy(d_B, B, B_rows * B_cols * sizeof(int), cudaMemcpyHostToDevice);

    dim3 dimBlock(BLOCK_SIZE, BLOCK_SIZE);
    dim3 dimGrid((C_cols + BLOCK_SIZE - 1) / BLOCK_SIZE, (C_rows + BLOCK_SIZE - 1) / BLOCK_SIZE);

    float gpuTime = 0, cpuTime = 0;
    cudaEvent_t start, stop;

    // GPU Timing
    cudaEventCreate(&start);
    cudaEventCreate(&stop);
    cudaEventRecord(start);

    matrix_multiply<<<dimGrid, dimBlock>>>(d_A, d_B, d_C, A_cols, C_rows, C_cols);

    cudaEventRecord(stop);
    cudaEventSynchronize(stop);
    cudaEventElapsedTime(&gpuTime, start, stop);
    cudaMemcpy(C, d_C, C_rows * C_cols * sizeof(int), cudaMemcpyDeviceToHost);

    cout << "GPU result:\n";
    // print_matrix(C, C_rows, C_cols);
    cout << "GPU Elapsed Time = " << gpuTime << " ms" << endl;

    // CPU Timing
    cudaEventRecord(start);
    matrix_multiplication_cpu(A, B, C, A_cols, C_rows, C_cols);
    cudaEventRecord(stop);
    cudaEventSynchronize(stop);
    cudaEventElapsedTime(&cpuTime, start, stop);

    cout << "CPU result:\n";
    // print_matrix(C, C_rows, C_cols);
    cout << "CPU Elapsed Time = " << cpuTime << " ms" << endl;

    cudaFree(d_A);
    cudaFree(d_B);
    cudaFree(d_C);
    delete[] A;
    delete[] B;
    delete[] C;

    return 0;
}


Overwriting matmul.cu


In [13]:
!nvcc -o matmul matmul.cu
!./matmul

Dimensions of matrix 1:
Rows: 300
Columns: 800
Dimensions of matrix 2:
Rows: 800
Columns: 250
GPU result:
GPU Elapsed Time = 0.170336 ms
CPU result:
CPU Elapsed Time = 242.038 ms


### Below is an inefficient code for the practical, so refer to the above codes

In [16]:
# %%writefile vector_Addition.cu
# #include<iostream>
# #include<bits/stdc++.h>
# #include<cuda.h>
# #define BLOCK_SIZE 16
# using namespace std;

# void fill_array(int *arr,int size){
#     for(int i = 0;i < size; i++){
#         arr[i] = rand() % 100;
#     }
# }

# void add_cpu(int *arr1, int *arr2, int *result, int size){
#     for(int i = 0;i < size; i++){
#         result[i] = arr1[i] + arr2[i];
#     }
# }

# void print_matrix(int *arr, int size){
#     for(int i = 0; i < size; i++){
#         cout << arr[i] << " ";
#     }
#     cout << endl;
# }

# __global__ void add(int *arr1, int *arr2, int *arr3,int size){
#     int block_id = blockIdx.x * blockDim.x + threadIdx.x;
#     if(block_id < size){
#         arr3[block_id] = arr1[block_id] + arr2[block_id];
#     }
# }

# int main(){
#     int *arr1_cpu,*arr2_cpu,*result_cpu;
#     int size;
#     cout << "Enter size of vector: ";
#     cin >> size;

#     arr1_cpu = new int[size];
#     arr2_cpu = new int[size];
#     result_cpu = new int[size];

#     fill_array(arr1_cpu,size);
#     cout << "Array 1: ";
#     print_matrix(arr1_cpu,size);
#     fill_array(arr2_cpu,size);
#     cout << "Array 2: ";
#     print_matrix(arr2_cpu,size);

#     int *arr1_gpu,*arr2_gpu,*result_gpu;

#     cudaMallocManaged(&arr1_gpu, size * sizeof(int));
#     cudaMallocManaged(&arr2_gpu, size * sizeof(int));
#     cudaMallocManaged(&result_gpu, size * sizeof(int));

#     cudaMemcpy(arr1_gpu,arr1_cpu,size * sizeof(int),cudaMemcpyHostToDevice);
#     cudaMemcpy(arr2_gpu,arr2_cpu,size * sizeof(int),cudaMemcpyHostToDevice);
#     cudaEvent_t start,stop;
#     float elapsedTime;

#     dim3 dimGrid(size + BLOCK_SIZE - 1 / BLOCK_SIZE);
#     dim3 dimBlock(BLOCK_SIZE);

#     cudaEventCreate(&start);
#     cudaEventCreate(&stop);
#     cudaEventRecord(start,0);

#     add<<<dimGrid,dimBlock>>>(arr1_gpu,arr2_gpu,result_gpu,size);
#     cudaEventRecord(stop,0);
#     cudaEventSynchronize(stop);
#     cudaEventElapsedTime(&elapsedTime,start,stop);
#     cudaEventDestroy(start);
#     cudaEventDestroy(stop);
#     cudaMemcpy(result_cpu,result_gpu,size * sizeof(int),cudaMemcpyDeviceToHost);
#     cout << "GPU result:\n";
#     print_matrix(result_cpu,size);
#     cout<<"Elapsed Time = "<<elapsedTime<<" milliseconds" << endl;
#     cudaFree(arr1_gpu);
#     cudaFree(arr2_gpu);
#     cudaFree(result_gpu);

#     cudaEventCreate(&start);
#     cudaEventCreate(&stop);
#     cudaEventRecord(start,0);

#     add_cpu(arr1_cpu,arr2_cpu,result_cpu,size);
#     cudaEventRecord(stop,0);
#     cudaEventSynchronize(stop);
#     cudaEventElapsedTime(&elapsedTime,start,stop);
#     cudaEventDestroy(start);
#     cudaEventDestroy(stop);
#     cout << "CPU result:\n";
#     print_matrix(result_cpu,size);
#     cout<<"Elapsed Time = "<<elapsedTime<<" milliseconds" << endl;

#     return 0;
# }

In [17]:
# !nvcc -o vector_Addition vector_Addition.cu
# !./vector_Addition

In [ ]:
# %%writefile matmul.cu

# #include<iostream>
# #include<bits/stdc++.h>
# #include<cuda.h>
# #define BLOCK_SIZE 16

# using namespace std;



# void initialize_matrix(int *array, int rows, int cols){
#     for(int i = 0 ; i < rows; i++){
#         for(int j = 0; j < cols; j++){
#             array[i*cols + j] = rand() % 10;
#         }
#     }
# }

# void print_matrix(int *array, int rows, int cols){
#     for(int i = 0 ; i < rows; i++){
#         for(int j = 0; j < cols; j++){
#             cout << array[i*cols + j] << " ";
#         }
#         cout << endl;
#     }
# }

# void matrix_multiplication_cpu(int *a, int *b, int *c, int common, int c_rows,int c_cols){
#     for(int i = 0; i < c_rows; i++){
#         for(int j = 0; j < c_cols; j++){
#             int sum = 0;
#             for(int k = 0; k < common; k++){
#                 sum += a[i*common + k] * b[k*c_cols + j];
#             }
#             c[i*c_cols + j] = sum;
#         }
#     }
# }



# __global__ void matrix_multiply(int *a, int *b, int *c, int c_rows, int common, int c_cols)
# {
#     int row = blockIdx.y*blockDim.y + threadIdx.y;
#     int col = blockIdx.x*blockDim.x + threadIdx.x;
#     int sum=0;

#     if(col < c_cols && row < c_rows) {
#       for(int j = 0 ;j < common;j++)
#       {
#           sum += a[row*common+j] * b[j*c_cols+col];
#       }
#       c[c_cols*row+col]=sum;
#     }

# }


# int main(){

#     int A_rows, A_cols, B_rows, B_cols, C_rows, C_cols;
#     cout << "Dimensions of matrix 1:\n";
#     cout << "Rows: ";
#     cin >> A_rows;
#     cout << "Columns: ";
#     cin >> A_cols;
#     cout << "Dimensions of matrix 2:\n";
#     cout << "Rows: " << A_cols << endl << "Columns: ";
#     cin >> B_cols;
#     B_rows = A_cols;
#     C_rows = A_rows;
#     C_cols = B_cols;

#     int A_size = A_rows * A_cols;
#     int B_size = B_rows * B_cols;
#     int C_size = C_rows * C_cols;

#     int *A, *B, *C;
#     int *m1,*m2,*result;

#     A = new int[A_size];
#     B = new int[B_size];
#     C = new int[C_size];

#     initialize_matrix(A,A_rows,A_cols);
#     cout << "Matrix 1\n";
#     print_matrix(A,A_rows,A_cols);
#     initialize_matrix(B,B_rows,B_cols);
#     cout << "Matrix 2\n";
#     print_matrix(B,B_rows,B_cols);

#     cudaMallocManaged(&m1, A_size * sizeof(int));
#     cudaMallocManaged(&m2, B_size * sizeof(int));
#     cudaMallocManaged(&result, C_size * sizeof(int));

#     cudaMemcpy(m1,A,A_size * sizeof(int), cudaMemcpyHostToDevice);
#     cudaMemcpy(m2,B,B_size * sizeof(int), cudaMemcpyHostToDevice);

#     dim3 dimGrid(A_rows + BLOCK_SIZE  - 1 / BLOCK_SIZE, B_cols + BLOCK_SIZE - 1 / BLOCK_SIZE);
#     dim3 dimBlock(BLOCK_SIZE,BLOCK_SIZE);

#     float gpu_elapsed_time;
#     cudaEvent_t gpu_start,gpu_stop;

#     cudaEventCreate(&gpu_start);
#     cudaEventCreate(&gpu_stop);
#     cudaEventRecord(gpu_start);
#     matrix_multiply<<<dimGrid,dimBlock>>>(m1,m2,result,C_rows,A_cols,C_cols);
#     cudaEventRecord(gpu_stop);
#     cudaEventSynchronize(gpu_stop);
#     cudaEventElapsedTime(&gpu_elapsed_time, gpu_start, gpu_stop);
#     cudaEventDestroy(gpu_start);
#     cudaEventDestroy(gpu_stop);

#     cudaMemcpy(C,result,C_size*sizeof(int),cudaMemcpyDeviceToHost);
#     cout << "GPU result:\n";
#     print_matrix(C,C_rows,C_cols);
#     cout<<"GPU Elapsed time is: "<<gpu_elapsed_time<<" milliseconds"<<endl;

#     cudaEventCreate(&gpu_start);
#     cudaEventCreate(&gpu_stop);
#     cudaEventRecord(gpu_start);
#     matrix_multiplication_cpu(A,B,C,A_cols,C_rows,C_cols);
#     cudaEventRecord(gpu_stop);
#     cudaEventSynchronize(gpu_stop);
#     cudaEventElapsedTime(&gpu_elapsed_time, gpu_start, gpu_stop);
#     cudaEventDestroy(gpu_start);
#     cudaEventDestroy(gpu_stop);

#     cout << "CPU result:\n";
#     print_matrix(C,C_rows,C_cols);
#     cout<<"CPU Elapsed time is: "<<gpu_elapsed_time<<" milliseconds"<<endl;

#     cudaFree(m1);
#     cudaFree(m2);
#     cudaFree(result);

#     return 0;
# }

In [ ]:
# !nvcc -o matmul matmul.cu
# !./matmul